In [9]:
import pandas as pd
from gurobipy import *

In [3]:
df = pd.read_csv("./avg_realbiopsies.csv", index_col="sample")
df = pd.read_csv("./2020_avg_interpolated.csv")
df = pd.read_csv("./new_fff.csv",index_col='sample')

In [13]:
df = pd.read_csv("./new_fff_ppp.csv",index_col=0)

In [14]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Interpolated,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# %%bash
# head ./avg_realbiopsies.csv

In [16]:
# remove where 9 is a child of 0

In [17]:
trees = [
#     {
# 	18: [19],
# 	19: [],
# 	7: [18, 17],
# 	17: [],
# 	4: [7],
# 	3: [15, 14],
# 	15: [],
# 	14: [],
# 	2: [13],
# 	13: [],
# 	1: [9],
# 	9: [],
# 	0: [16, 12, 11, 10, 8, 6, 5, 4, 3, 2, 1],
# 	16: [],
# 	12: [],
# 	11: [],
# 	10: [],
# 	8: [],
# 	6: [],
# 	5: []
# }, 
    {
	18: [19],
	19: [],
	7: [18, 17, 16],
	17: [],
	16: [],
	4: [7],
	3: [15, 14],
	15: [],
	14: [],
	2: [13],
	13: [],
	1: [9],
	9: [],
	0: [12, 11, 10, 8, 6, 5, 4, 3, 2, 1],
	12: [],
	11: [],
	10: [],
	8: [],
	6: [],
	5: []
}, 
#     {
# 	18: [19],
# 	19: [],
# 	7: [18, 17],
# 	17: [],
# 	4: [7],
# 	3: [15, 14],
# 	15: [],
# 	14: [],
# 	2: [13],
# 	13: [],
# 	1: [9, 8],
# 	9: [],
# 	8: [],
# 	0: [16, 12, 11, 10, 6, 5, 4, 3, 2, 1],
# 	16: [],
# 	12: [],
# 	11: [],
# 	10: [],
# 	6: [],
# 	5: []
# }, 
    # original below
    {
	18: [19],
	19: [],
	7: [18, 17, 16],
	17: [],
	16: [],
	4: [7],
	3: [15, 14],
	15: [],
	14: [],
	2: [13],
	13: [],
	1: [9, 8],
	9: [],
	8: [],
	0: [12, 11, 10, 6, 5, 4, 3, 2, 1],
	12: [],
	11: [],
	10: [],
	6: [],
	5: []
}, ]

In [18]:
model = Model("tree")
m = len(df)
n = len(df.columns)

f = [ [model.addVar(vtype=GRB.CONTINUOUS, name="f_%d_%d" % (p, i)) for i in range(n)] for p in range(m)]
g = [ [model.addVar(vtype=GRB.CONTINUOUS, name="g_%d_%d" % (p, i)) for i in range(n)] for p in range(m)]
f_hat = [ [df.iloc[p][i] for i in range(n) ] for p in range(m) ]

for p in range(m):
    for i in range(n):
        model.addConstr(0 <= f[p][i])
        model.addConstr(f[p][i] <= 1)
        model.addConstr(0 <= g[p][i])
        model.addConstr(g[p][i] <= 1)
#         if i in leaf:
#             model.addConstr(f[p][i] == f_hat[p][i])
        if i > 0:
            model.addConstr(g[p][i] >= f[p][i] - f_hat[p][i])
            model.addConstr(g[p][i] >= f_hat[p][i] - f[p][i])
            
        elif max(f_hat[p]) >= 0.01:
            model.addConstr(f[p][i] >= 0.7)
            
        for delta in trees: 
            rhs = quicksum([f[p][j] for j in delta[i]])
            if i > 0:
                model.addConstr(f[p][i] >= rhs)
            else:
                model.addConstr(f[p][i] >= rhs)
model.setObjective(quicksum(f[p][0] + g[p][i] for p in range(m) for i in range(1,n)), GRB.MINIMIZE)

In [19]:
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 293025 rows, 73920 columns and 433473 nonzeros
Model fingerprint: 0x2d8afc03
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 69309 rows and 282227 columns
Presolve time: 1.14s
Presolved: 4611 rows, 10798 columns, 16161 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.805e+03
 Factor NZ  : 1.494e+04 (roughly 6 MBytes of memory)
 Factor Ops : 6.963e+04 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 1.23 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 2698 iterations and 1.30 seconds
Optimal objective  1.396767413e+04


In [20]:
with open("new_fff_ppp_lp_2trees_nobaz.csv", "w") as ff:
    for p in range(m):
        ff.write(",".join(map(str, [f[p][i].x for i in range(n)])) + "\n")